In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import fastai
from fastai import *
from fastai.vision import *
from upassai import *
fastai.__version__

'1.0.39'

In [10]:
vggface2_path = '../data/vggface2'

## prepare valid dataset

In [31]:
valid_df = pd.read_csv('dataset/valid_result.csv')

In [36]:
valid_source = create_bunch(valid_df, cols='source',
                            path=vggface2_path+'/valid/')
valid_target = create_bunch(valid_df, cols='target',
                            path=vggface2_path+'/valid/')
# valid_source.transform(get_transforms())

In [37]:
valid_ds = TwinDataset(valid_source.train_ds, valid_target.train_ds)

## prepare train dataset

In [38]:
train_df = pd.read_csv('dataset/train_result.csv')

In [39]:
train_source = create_bunch(train_df, cols='source',
                            path=vggface2_path+'/train/')
train_target = create_bunch(train_df, cols='target',
                            path=vggface2_path+'/train/')

In [40]:
train_ds = TwinDataset(train_source.train_ds, train_target.train_ds)

## final databunch

In [41]:
twin_databunch = DataBunch.create(
    train_ds=train_ds, valid_ds=valid_ds, path=vggface2_path, bs=4)

## training

In [53]:
learner = Learner(twin_databunch,
                  SiameseNetwork(
                      architecture=fastai.vision.models.resnet152,
                      nf=4096, nc=2,
                      lin_ftrs=[800, 500, 300, 100, 50, 20]),
                  loss_func=contrastive_loss, path='./models')
learner

Learner(data=DataBunch;

Train: <upassai.data.TwinDataset object at 0x13a7a0080>;

Valid: <upassai.data.TwinDataset object at 0x13a7a0048>;

Test: None, model=SiameseNetwork(
  (body): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, e

In [54]:
learner.fit_one_cycle(1)

epoch,train_loss,valid_loss


Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/upass/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/local/anaconda3/envs/upass/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/local/anaconda3/envs/upass/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/local/anaconda3/envs/upass/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 